### Q1. Running Elastic
1. Run Elastic Search with docker
2. Get the cluster information  with curl localhost:9200

### Import libraries, modules and packages

In [1]:
import requests 
import os
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
import tiktoken
from dotenv import load_dotenv
from openai import OpenAI

/opt/conda/envs/llm-intro/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
# Load the .env file
load_dotenv()

# Get the API key from the environment variable
openai_api_key = os.getenv("OPENAI_API_KEY")

### Getting the data

In [3]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

### Q2. Indexing the data

In [4]:
es_client = Elasticsearch('http://localhost:9200') 

In [5]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} # primary key in RDBMS taste
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

### Filtering and Searching

In [6]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:19<00:00, 49.88it/s]


In [7]:
def elastic_search(query, size=5, search_words=[]):
    def filter_search():
        return list(map(lambda word: {"term": {**word}}, search_words))
        
    search_query = {
        "size": size,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": filter_search()
            }
        }
    }

    return es_client.search(index=index_name, body=search_query)

In [8]:
query = "How do I execute a command in a running docker container?"

### Q3. Searching -  the score for the top ranking result

In [9]:
print(f"The top-ranking result score is {elastic_search(query)['hits']['max_score']}")

The top-ranking result score is 84.050095


### Q4. Filtering - the 3rd question returned by the search engine

In [12]:
results = elastic_search(query, 3, [{"course": "machine-learning-zoomcamp"}])["hits"]["hits"]

for doc in results: 
  print(doc["_source"]["question"])

How do I debug a docker container?
How do I copy files from my local machine to docker container?
How do I copy files from a different folder into docker container’s working directory?


In [14]:
print(f"The 3rd question returned by the search engine is: {results[2]['_source']['question']}")

The 3rd question returned by the search engine is: How do I copy files from a different folder into docker container’s working directory?


### Q5. Building a prompt - the length of the prompt

In [15]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [16]:
context = ""
for doc in results:
  context += context_template.format(question=doc["_source"]["question"], text=doc["_source"]["text"]) + "\n\n"

print(context)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [17]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [18]:
prompt = prompt_template.format(question=query, context=context).strip()
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

In [19]:
print(f"The length of the prompt is {len(prompt)}")

The length of the prompt is 1462


### Q6. Convert the prompt into input vectors (tokens) for the language model.
* Calculate the number of tokens

In [20]:
encoding = tiktoken.encoding_for_model("gpt-4o")

encoded_prompt = encoding.encode(prompt)
print(f"The number of tokens we send in our prompt is {len(encoded_prompt)}")

The number of tokens we send in our prompt is 322


### Generate response (answer) to the question with LLM 

In [24]:
def ask_llm(prompt, model="gpt-4o"):
    client = OpenAI(api_key=openai_api_key)
    
    chat_completion = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model=model,
    )

    return chat_completion.choices[0].message.content

In [25]:
response = ask_llm(prompt)
print(response)

To execute a command in a running Docker container, you should use the `docker exec` command. Here are the steps:

1. First, find the container ID of the running container by using the `docker ps` command.
2. Once you have the container ID, use the following command to execute a bash shell (or any other command) in the running container:
   ```sh
   docker exec -it <container-id> bash
   ```

Simply replace `<container-id>` with the actual container ID you obtained from the `docker ps` output.

For example:
```sh
docker ps
docker exec -it abc123def456 bash
```


### Calculating the costs

In [26]:
def calculate_price(input_tokens: int, output_tokens: int, number_of_api_calls: int):
    # input and output pricing
    input_price = 0.005 * input_tokens * number_of_api_calls / 1e3
    output_price = 0.015 * output_tokens * number_of_api_calls / 1e3
    
    # Total price
    total_price = input_price + output_price 
    
    print(f'Input price: ${input_price:.4f}')
    print(f'Output price: ${output_price:.4f}')
    print(f'The total price is: ${total_price:.4f}')
    
    return total_price

# Small-scale application
requests_number = 1000

print("\nThe actual cost of the encoded lengths:")
calculate_price(len(encoded_prompt), len(encoding.encode(response)), requests_number)


The actual cost of the encoded lengths:
Input price: $1.6100
Output price: $2.0100
The total price is: $3.6200


3.62